In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import TheilSenRegressor
from pyscipopt import Model, quicksum, multidict
import os
os.chdir('/Users/user/PycharmProjects/Shell_hachakton_2023')

In [2]:
data = pd.read_csv('data/Biomass_History.csv', index_col=0)
dist = pd.read_csv('data/Distance_Matrix.csv', index_col=0)

In [52]:
def predict_biomass(data):
    reg = TheilSenRegressor()
    X = np.arange(0,8).reshape(-1,1)
    pred_list = []
    for i in range(0, len(data)):
        y = data.iloc[i, 2:].values.ravel()
        reg.fit(X,y)
        preds = reg.predict(np.array([8, 9]).reshape(-1, 1))
        pred_list.append(list(preds))
    pred_df = pd.DataFrame(np.array(pred_list).squeeze(), columns=['2018', '2019'],
                               index=np.arange(0, len(data)))
    data_predicted = pd.concat([data, pred_df], axis=1)

    return data_predicted

In [3]:
def CFLP(I,J,d,M,c,n):
    model = Model("flp")
    x,y = {},{}

    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i) # All demands should be met
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i) # All facility should not violate capacity constraint
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j)) # Can't be delivered more than demand
    model.addCons(quicksum(y[j] for j in J) <= n)
    model.setObjective(
        quicksum((M[j]*y[j] - quicksum(x[i,j] for i in I)) for j in J) +
        0.001 * quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [4]:
def CFLP_second_year(I,J,d,M,c,y):
    model = Model("flp")
    x = {}

    for j in J:
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i) # All demands should be met
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i) # All facility should not violate capacity constraint
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j)) # Can't be delivered more than demand
    model.setObjective(
        quicksum((M[j]*y[j] - quicksum(x[i,j] for i in I)) for j in J) +
        0.001 * quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x
    return model

### Prediction

In [53]:
prediction = predict_biomass(data)
prediction

,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,24.66818,71.33144,8.475744,8.868568,9.202181,6.023070,10.788374,6.647325,7.387925,5.180296,6.941737,6.691071
1,24.66818,71.41106,24.029778,28.551348,25.866415,21.634459,34.419411,27.361908,40.431847,42.126945,43.204805,45.605177
2,24.66818,71.49069,44.831635,66.111168,56.982258,53.003735,70.917908,42.517117,59.181629,73.203232,72.864142,75.675037
3,24.66818,71.57031,59.974419,80.821304,78.956543,63.160561,93.513924,70.203171,74.536720,101.067352,98.593218,102.801545
4,24.66818,71.64994,14.653370,19.327524,21.928144,17.899586,19.534035,19.165791,16.531315,26.086885,23.437156,24.141703
...,...,...,...,...,...,...,...,...,...,...,...,...
2413,20.15456,72.84432,5.199882,4.516778,4.321080,2.658953,5.113997,5.301668,6.419223,5.321604,5.509466,5.651458
2414,20.15456,72.92394,0.122287,0.126717,0.101494,0.111509,0.121749,0.122467,0.145785,0.120626,0.124472,0.125093
2415,20.15456,73.00357,0.039415,0.040843,0.032713,0.035941,0.039241,0.039473,0.046989,0.038879,0.040218,0.040564
2416,20.15456,73.08319,2.719220,1.370163,0.818687,1.205721,1.316443,1.324201,1.576338,1.304297,1.265245,1.253341


### First year 2018

In [59]:
year = '2018'
n = 25
cap = 5000

biomass = prediction[year].sample(100)
I, d = multidict(dict(biomass))
capacity = pd.Series(cap, index=biomass.index)
J, M = multidict(dict(capacity))
c = dist.iloc[biomass.index, biomass.index].reset_index().melt(id_vars='index')
c['variable'] = c['variable'].astype('int')
c = {(row[0], row[1]):row[2] for row in c.values}
d

{940: 238.4532218191114,
 2345: 38.33680492832701,
 939: 243.4703203535991,
 2261: 80.44507022058764,
 1794: 192.11508688867653,
 1687: 339.5967163721259,
 1612: 182.57259866675645,
 426: 2.433090261422842,
 1230: 249.58900429670973,
 1684: 166.65067313733738,
 1311: 1.5389039715129678,
 230: 272.3014730179773,
 74: 194.45025953170608,
 2030: 281.4124043244808,
 876: 276.30544738598394,
 811: 217.7536549195849,
 387: 124.78515026243156,
 724: 0.12961485794145608,
 1739: 33.02252968482857,
 503: 212.2748221860799,
 1525: 127.32861303322919,
 1859: 369.6830387118906,
 718: 5.929010127021876,
 2036: 164.4247527136379,
 365: 11.89916315766998,
 1099: 228.28166055333332,
 2180: 0.21045543187373475,
 1082: 17.867498367017525,
 715: 19.71710109362552,
 867: 200.64065531958357,
 1679: 230.99690149392262,
 1176: 299.1073971556888,
 2017: 208.04601430771203,
 1164: 61.19399666394661,
 2361: 0.03470861471010189,
 127: 137.86242500555858,
 2390: 45.310903359529334,
 1244: 31.589321875846345,
 15: 

In [ ]:
model = CFLP(I, J, d, M, c, n)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print("Optimal value=", model.getObjVal())
print("Facilities at nodes:", facilities)
print("Edges:", edges)

### Save locations

In [26]:
boolean_depot_location = {j:model.getVal(y[j]) for j in J}
boolean_depot_location

{1570: 0.0,
 2297: 0.0,
 574: 0.0,
 1672: 0.0,
 1714: 0.0,
 2103: 0.0,
 1175: 0.0,
 692: 0.0,
 1922: 1.0,
 368: 0.0}

### Second year 2019

In [27]:
indices = biomass.index
biomass = biomass_2011[indices]
I, d = multidict(dict(biomass))
capacity = pd.Series(20000, index=biomass.index)
J, M = multidict(dict(capacity))
c = dist.iloc[biomass.index, biomass.index].reset_index().melt(id_vars='index')
c['variable'] = c['variable'].astype('int')
c = {(row[0], row[1]):row[2] for row in c.values}
d

{1570: 266.0648193,
 2297: 134.6060333,
 574: 9.033880234,
 1672: 159.418869,
 1714: 226.333725,
 2103: 34.51924896,
 1175: 126.8894653,
 692: 61.23438263,
 1922: 446.1592407,
 368: 98.30596924}

In [30]:
model = CFLP_second_year(I, J, d, M, c, y=boolean_depot_location)
model.optimize()
EPS = 1.e-6
x = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
print("Optimal value=", model.getObjVal())
print("Edges:", edges)

Optimal value= 18785.259442601044
Edges: [(1570, 1922), (2297, 1922), (574, 1922), (1672, 1922), (1714, 1922), (2103, 1922), (1175, 1922), (692, 1922), (1922, 1922), (368, 1922)]


In [10]:
year = 2018
data_type = 'biomass_demand_supply'

def get_routes_solution(model, edges, year, data_type):
    solution = np.zeros(shape=(len(edges), 3))
    for i,edge in enumerate(edges):
        solution[i,0] = edge[0]
        solution[i,1] = edge[1]
        solution[i,2] = model.getVal(x[edge])
    solution = pd.DataFrame(solution, columns=['source_index', 'destination_index', 'value'])
    solution['year'] = year
    solution['data_type'] = data_type
    solution['source_index'] = solution['source_index'].astype('int')
    solution['destination_index'] = solution['destination_index'].astype('int')
    solution = solution[['year', 'data_type', 'source_index', 'destination_index', 'value']]
    
    return solution

def get_locations(model, facilities, year, data_type):

    solution = pd.DataFrame(np.zeros(shape=(len(facilities), 5)), columns = ['year', 'data_type', 'source_index', 'destination_index', 'value'])
    solution['year'] = year; solution['data_type'] = data_type
    solution['destination_index'] = np.nan; solution['value'] = np.nan
    solution['source_index'] = facilities

    return solution



In [ ]:
get_routes_solution(model, edges, year, data_type)

In [11]:
get_locations(model, facilities, year, data_type)

,year,data_type,source_index,destination_index,value
0,2018,biomass_demand_supply,2,NaN,NaN
1,2018,biomass_demand_supply,3,NaN,NaN
